In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("loan_default.csv")

In [3]:
data.head(10)

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,DAYS_BIRTH,YEARS_EMPLOYED
0,0.0,Cash loans,M,Y,N,1,225000.0,578619.0,23229.0,-12347,0
1,0.0,Revolving loans,M,Y,Y,1,NaN,270000.0,13500.0,-14048,6
2,0.0,Cash loans,M,Y,N,0,144000.0,753840.0,29340.0,-14639,6
3,0.0,Cash loans,F,N,Y,0,81000.0,98910.0,7785.0,-14591,11
4,0.0,Cash loans,F,N,Y,1,103500.0,521280.0,26779.5,-12023,0
5,1.0,Cash loans,M,Y,Y,2,202500.0,781920.0,40054.5,-11653,3
6,0.0,Cash loans,F,N,Y,1,NaN,305221.5,20002.5,-11665,2
7,0.0,Cash loans,F,Y,N,0,202500.0,315000.0,13873.5,-10465,5
8,0.0,Cash loans,M,Y,Y,0,337500.0,493497.0,58698.0,-10136,3
9,1.0,Cash loans,M,Y,Y,0,202500.0,254700.0,25834.5,-9785,3


In [4]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
TARGET,10281.0,0.079078,0.269873,0.0,0.000,0.0,0.0,1.0
CNT_CHILDREN,10284.0,10.287340,986.138233,0.0,0.000,0.0,1.0,99999.0
AMT_INCOME_TOTAL,6993.0,167909.947902,105245.713874,27000.0,112500.000,144000.0,202500.0,3825000.0
AMT_CREDIT,10284.0,599902.578326,400843.758473,45000.0,270000.000,512446.5,810000.0,4050000.0
AMT_ANNUITY,10284.0,27191.230747,14933.326225,2974.5,16509.375,25128.0,34596.0,258025.5
DAYS_BIRTH,10284.0,-16055.232400,4377.738214,-25166.0,-19724.250,-15733.5,-12352.0,-7680.0
YEARS_EMPLOYED,10284.0,-176.848016,388.125395,-1000.0,0.000,3.0,7.0,150.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10284 entries, 0 to 10283
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   TARGET              10281 non-null  float64
 1   NAME_CONTRACT_TYPE  10284 non-null  object 
 2   CODE_GENDER         10284 non-null  object 
 3   FLAG_OWN_CAR        8022 non-null   object 
 4   FLAG_OWN_REALTY     10254 non-null  object 
 5   CNT_CHILDREN        10284 non-null  int64  
 6   AMT_INCOME_TOTAL    6993 non-null   float64
 7   AMT_CREDIT          10284 non-null  float64
 8   AMT_ANNUITY         10284 non-null  float64
 9   DAYS_BIRTH          10284 non-null  int64  
 10  YEARS_EMPLOYED      10284 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 883.9+ KB


In [6]:
data.isna().sum()

TARGET                   3
NAME_CONTRACT_TYPE       0
CODE_GENDER              0
FLAG_OWN_CAR          2262
FLAG_OWN_REALTY         30
CNT_CHILDREN             0
AMT_INCOME_TOTAL      3291
AMT_CREDIT               0
AMT_ANNUITY              0
DAYS_BIRTH               0
YEARS_EMPLOYED           0
dtype: int64

In [7]:
data.nunique()

TARGET                   2
NAME_CONTRACT_TYPE       2
CODE_GENDER              2
FLAG_OWN_CAR             2
FLAG_OWN_REALTY          4
CNT_CHILDREN            13
AMT_INCOME_TOTAL       260
AMT_CREDIT            1860
AMT_ANNUITY           4279
DAYS_BIRTH            7604
YEARS_EMPLOYED          46
dtype: int64

# Treating Inconsistencies

In [8]:
data["FLAG_OWN_REALTY"].unique()

array(['N', 'Y', nan, 'YES', 'NO'], dtype=object)

In [12]:
data["FLAG_OWN_REALTY"] = data["FLAG_OWN_REALTY"].replace({"YES": "Y", "NO": "N"})
data["FLAG_OWN_REALTY"].unique()

array(['N', 'Y', nan], dtype=object)

# Treating Missing Values

The TARGET column, which is our dependent variable, has 3 missing entries. Given the small number of missing values relative to the dataset size, and the importance of having complete labels for all training instances in supervised learning, the most appropriate action is to drop these rows.

In [28]:
# Drop rows where 'TARGET' is missing
data.dropna(subset=["TARGET"], inplace=True) 

The FLAG_OWN_CAR column, which indicates whether the applicant owns a car, contains 2,262 missing entries. Since this is a categorical variable, we will impute the missing values using the mode of this column, representing the most common category.

In [27]:
# Impute missing values in 'FLAG_OWN_CAR' with the mode
car_mode = data["FLAG_OWN_CAR"].mode()[0]
data["FLAG_OWN_CAR"] = data["FLAG_OWN_CAR"].fillna(car_mode)

For the AMT_INCOME_TOTAL column, which represents the total income of the applicant and has 3,291 missing values, we will use the median to impute these values. The median is preferred over the mean as it is more robust to outliers, which are evident from the wide range of income values present in our dataset

In [25]:
# Impute missing values in 'AMT_INCOME_TOTAL' with the median
income_median = data["AMT_INCOME_TOTAL"].median()
data["AMT_INCOME_TOTAL"] = data["AMT_INCOME_TOTAL"].fillna(income_median)

In [29]:
# Display the summary to verify changes
data.isna().sum()

TARGET                 0
NAME_CONTRACT_TYPE     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY       30
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
AMT_CREDIT             0
AMT_ANNUITY            0
DAYS_BIRTH             0
YEARS_EMPLOYED         0
dtype: int64